In [298]:
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
import requests

In [299]:
base_url = 'https://www.darkhorse.com/'
start_url = 'https://www.darkhorse.com/Comics/Browse/January+1986-December+2020---0-Z/P5wfwkt8?page='


Start with a base URL, in this case just www.darkhorse.com, and a start_url, in this case where the browse section starts on page 1.  Since different comics have different contributers (artists,writers, inkers etc) we need to generate a base list of attribute we will collect, so we loop through the first page and create a list of attributes. 

In [300]:
spec_url = 'https://www.darkhorse.com/Comics/Browse/January+1986-December+2020---0-Z/P5wfwkt8?page=1'
html_titles = requests.get(spec_url).text
titles = BeautifulSoup(html_titles, 'lxml')
prod_list = titles.find_all('div',{"class":"list_item"})
page_links = [prod_list[i].select('a')[1]['href'] for i in range(len(prod_list))]

base_specs = []
for link in page_links:
        request = requests.get(base_url + link).text
        soup = BeautifulSoup(request, 'lxml')
        for i in soup.find_all('dt'):
            if i.text[:-1] not in base_specs:
                base_specs.append(i.text[:-1])

Now that we have a base list of specifications/attributes for a comic, we need to loop through each book on each page of the site the get the info we want. 

In [301]:
page_n = 1
books = []
while True:
    html_titles = requests.get(start_url+str(page_n)).text #get page 'i'
    titles = BeautifulSoup(html_titles, 'lxml') #soupify
    prod_list = titles.find_all('div',{"class":"list_item"}) #get list of titles
    if not prod_list:
        print('No more results: Exiting...') #check to see if results exits, other wise break
        break
    else:
        print('Getting page:{}'.format(page_n))
        
        page_links = [prod_list[i].select('a')[1]['href'] for i in range(len(prod_list))] #list of links
        for link in page_links: #iterate through links)
            data = []
            request = requests.get(base_url + link).text
            soup = BeautifulSoup(request, 'lxml')            
            data.append(link.split('/')[2]) #append job id
            data.append(soup.find_all('h2',{'class':'title'})[0].text) #append title
            data.append(soup.find_all('div',{'class':'product-description'})[0].text) #append description
            spec = [i.text[:-1] for i in soup.find_all('dt')] #generate list of specs
            res = [i.text for i in soup.find_all('dd')] #generate spec details
            d = {i:j for i,j in zip(spec,res)}
            for i in base_specs: #iterate through base_specs; if present add to data, else append 'Na'
                if i in d.keys():
                    data.append(d[i])
                else:
                    data.append('Na')
            
            books.append(data) #append book data to list of books       
    page_n += 1 #go to next page

Getting page:1
Getting page:2
Getting page:3
Getting page:4
Getting page:5
Getting page:6
Getting page:7
Getting page:8
Getting page:9
Getting page:10
Getting page:11
Getting page:12
Getting page:13
Getting page:14
Getting page:15
Getting page:16
Getting page:17
Getting page:18
Getting page:19
Getting page:20
Getting page:21
Getting page:22
Getting page:23
Getting page:24
Getting page:25
Getting page:26
Getting page:27
Getting page:28
Getting page:29
Getting page:30
Getting page:31
Getting page:32
Getting page:33
Getting page:34
Getting page:35
Getting page:36
Getting page:37
Getting page:38
Getting page:39
Getting page:40
Getting page:41
Getting page:42
Getting page:43
Getting page:44
Getting page:45
Getting page:46
Getting page:47
Getting page:48
Getting page:49
Getting page:50
Getting page:51
Getting page:52
Getting page:53
Getting page:54
Getting page:55
Getting page:56
Getting page:57
Getting page:58
Getting page:59
Getting page:60
Getting page:61
Getting page:62
Getting page:63
G

'books' is a list of lists, each list containing the desired information. We can use pandas to house the info in a DataFrame.

In [338]:
books = pd.DataFrame.from_records(books)
cols = ['Job#','Title','Description'] + base_specs
books.columns = cols
books['Description'] = [i.replace('\n','') for i in books['Description']]
books


,Job#,Title,Description,Writer,Artist,Colorist,Cover Artist,Publication Date,Format,Price,UPC,Editor,Letterer,Penciller,Inker
0,3004-394,Frankenstein Undone #3,Pursuing his humanity in the unlikely company ...,"Mike Mignola, Scott Allie",Ben Stenbeck,Brennan Wagner,Ben Stenbeck,"March 25, 2020","FC, 32 pages; Miniseries",$3.99,7 61568 00538 7 00311,Na,Na,Na,Na
1,3004-399,Frankenstein Undone #3 (Steve Purcell Variant ...,Pursuing his humanity in the unlikely company ...,"Mike Mignola, Scott Allie",Ben Stenbeck,Brennan Wagner,Steve Purcell,"March 25, 2020","FC, 32 pages; Miniseries",$3.99,7 61568 00538 7 00321,Na,Na,Na,Na
2,3006-241,Hidden Society #2,A new series from Rafael Albuquerque and Rafae...,Rafael Scavone,Rafael Albuquerque,Marcelo Costa,Rafael Albuquerque,"March 25, 2020","FC, 32 pages; Miniseries",$3.99,7 61568 00626 1 00211,Na,Na,Na,Na
3,3006-813,Hidden Society #2 (Greg Tocchini Variant Cover),A new series from Rafael Albuquerque and Rafae...,Rafael Scavone,Rafael Albuquerque,Marcelo Costa,Greg Tocchini,"March 25, 2020","FC, 32 pages; Miniseries",$3.99,7 61568 00626 1 00221,Na,Na,Na,Na
4,3005-923,Tomorrow #2,Time Out--ForeverOscar starts on the arduous p...,Peter Milligan,Jesus Hervas,James Devlin,Jesus Hervas,"March 25, 2020","FC, 32 pages; Miniseries",$3.99,7 61568 00627 8 00211,Karen Berger,Na,Na,Na
5,3004-660,Witchfinder: The Reign of Darkness #5,Sir Edward Grey and Sarah Jewell prepare to fa...,"Mike Mignola, Chris Roberson",Christopher Mitten,Michelle Madsen,Christopher Mitten,"March 25, 2020","FC, 32 pages; Miniseries",$3.99,7 61568 00576 9 00511,Na,Na,Na,Na
6,3000-872,X-RAY ROBOT #1,Max is a family man seeking a more interesting...,Michael Allred,Michael Allred,Laura Allred,Michael Allred,"March 25, 2020","FC, 32 pages; Miniseries",$3.99,7 61568 00657 5 00111,Na,Na,Na,Na
7,3001-571,X-RAY ROBOT #1 (Chris Samnee & Matthew Wilson ...,Max is a family man seeking a more interesting...,Michael Allred,Michael Allred,Laura Allred,"Chris Samnee, Matthew Wilson","March 25, 2020","FC, 32 pages; Miniseries",$3.99,7 61568 00657 5 00121,Na,Na,Na,Na
8,3005-795,BANG! #2,Matt Kindt! Wilfredo Torres!A constant stream ...,Matt Kindt,Wilfredo Torres,Nayoung Kim,Wilfredo Torres,"March 18, 2020","FC, 32 pages; Miniseries",$3.99,7 61568 00624 7 00211,Na,Nate Piekos,Na,Na
9,3006-612,BANG! #2 (Matt Kindt Variant Cover),Matt Kindt! Wilfredo Torres!A constant stream ...,Matt Kindt,Wilfredo Torres,Nayoung Kim,Matt Kindt,"March 18, 2020","FC, 32 pages; Miniseries",$3.99,7 61568 00624 7 00221,Na,Nate Piekos,Na,Na


In [337]:
books.to_csv('DH_archive.csv')